# 开放平台视频生成批量操作

**This tutorial is Only in Chinese explanation**

本项目是基于开放平台的视频生成批量操作，主要包括视频生成功能，通过本demo你可以快速生成多个一致性高度相同的视频备选素材。

提交包含 `input_text`, `image_path` 字段的 `level_contexts.xlsx` 文件

`input_text`：文本内容 

`image_path`: 图片绝对路径

输出结果为视频任务 id 的 csv文件，

## 1. Set up the environment 

首先，我们需要安装最新版本的智谱AI SDK，只有 > 2.1.4 的SDK版本支持视频生成模型。 同时，我们设定好 API Key。

In [6]:
!pip install zhipuai-platform-video -U

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os

os.environ["ZHIPUAI_API_KEY"] = "your zhipuai api key"

# 2.准备数据

基础表格里需要准备的数据包含了 `input_text`, `image_path` 字段，
`input_text` 为生成提示词，用于控制镜头以及
`image_path` 为起始帧的图片，这个图片可以不存在，使用 `request_img` 命令参数控制是否必须采用起始帧，默认必须存在


In [11]:
import pandas as pd
import os
from pathlib import Path
 
sample_data_dir = os.path.join(
    Path().cwd(), "input/image"
)
init_dataset = pd.DataFrame([
    {
        "input_text": "写实风格，风吹动女孩的头发，女孩悲伤的看着前方@@1.jpg",
        "image_path": os.path.join(sample_data_dir,"1.png")
    },
    {
        "input_text": "写实风格，女孩转头看向左方@@2.jpg",
        "image_path":  os.path.join(sample_data_dir,"1.png")
    },
    {
        "input_text": "写实风格，女人眨眼睛，眼睛中有漂亮的光影流转@@3.jpg",
        "image_path":  os.path.join(sample_data_dir,"1.png")
    },
    {
        "input_text": "写实风格，太阳缓缓升起，女人眨眼睛@@4.jpg",
        "image_path":  os.path.join(sample_data_dir,"1.png")
    },
    {
        "input_text": "写实风格，女人眨眼睛，远处的太阳逐渐升起@@5.jpg",
        "image_path":  os.path.join(sample_data_dir,"1.png")
    }
])


sample_data_dir = os.path.join(
    Path().cwd(), "input/"
)
if not os.path.exists(sample_data_dir):
    os.makedirs(sample_data_dir)
init_dataset.to_excel(f'{sample_data_dir}/dataset.xlsx', index=False)
sample_data_dir

'/media/gpt4-pdf-chatbot-langchain/glm-cookbook/vision/input/'


# 3. 运行

**由于平台没有当前任务进度的获取方式**

**这里会出现1302错误，你可以选择多尝试几次，继续任务**

**或者联系商务**

在同一个目录下可以断点继续任务，会自动跳过已经生成的任务, 下面是参数说明

**input_excel**：为包含input_text,image_path字段的的level_contexts.xlsx文件

**output_path**：为输出文件路径，生成 video_report.csv文件

根据需要配置线程
**request_img**： 是否必须采用起始帧。true,false

**prompt_num_threads**: 提示词线程数,默认2

**video_num_threads**: 视频生成线程数,默认1


In [14]:
output_data_dir = os.path.join(
    Path().cwd(), "output/"
)
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)
    
# 定义变量
input_excel = f'{sample_data_dir}/dataset.xlsx'
output_path = output_data_dir

# 使用 f-string 将变量传递给系统命令
command = f"python -m zhipuai_platform_video.start --input_excel {input_excel} --output_path {output_path}"
!{command}

Running VideoStrategyGenerator:VideoStrategyGenerator-4ab9ed565bc98ceebb772abe56ab0319 None
2024-07-29 14:59:08,376 - httpx - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/videos/generations "HTTP/1.1 200 OK"
Running VideoStrategyGenerator:VideoStrategyGenerator-ec931bb2f04fd3e79318ec343564ea87 None
2024-07-29 14:59:16,088 - httpx - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/videos/generations "HTTP/1.1 200 OK"


# 4. 提示词改写

程序会对input_text的内容进行改写，通过prompt_report文件可以查看效果，如果你觉得效果不好，删除**cache_data/VideoStrategyPrompt**文件夹可重试

In [27]:
prompt_report_csv = f'{output_data_dir}/prompt_report.csv'
prompt_report = pd.read_csv(prompt_report_csv)
prompt_report

,input_text,image_path,video_prompt
0,写实风格，风吹动女孩的头发，女孩悲伤的看着前方@@1.jpg,/media/gpt4-pdf-chatbot-langchain/glm-cookbook...,在一个静谧的午后，写实风格的镜头捕捉到了这样一个画面：微风轻轻拂过，带着夏日尾声的清爽，轻轻...
1,写实风格，女孩转头看向左方@@2.jpg,/media/gpt4-pdf-chatbot-langchain/glm-cookbook...,在这个写实风格的场景中，我们可以这样描述女孩的动作：\n\n女孩静静地站在那里，突然，她的头...
2,写实风格，女人眨眼睛，眼睛中有漂亮的光影流转@@3.jpg,/media/gpt4-pdf-chatbot-langchain/glm-cookbook...,在那个静谧的午后，阳光透过窗户洒在室内的角落，形成斑驳的光影。女人轻轻地眨了眨眼睛，随着眼睑...
3,写实风格，太阳缓缓升起，女人眨眼睛@@4.jpg,/media/gpt4-pdf-chatbot-langchain/glm-cookbook...,在这个写实风格的场景中，画面呈现的是清晨时分，太阳刚刚从地平线缓缓升起，阳光逐渐洒满大地。以...
4,写实风格，女人眨眼睛，远处的太阳逐渐升起@@5.jpg,/media/gpt4-pdf-chatbot-langchain/glm-cookbook...,在这个场景中，一位超写实画家正专注于描绘闭着眼睛的女人的肖像画。画家的每一笔每一划都非常细腻...


# 5.获取任务结果 
上方数据执行完成之后会生成video_report文件，其中包含任务id，这里可以根据id拉取视频链接

**出现Error processing video_task_id:任务可能在排队处理，多尝试几次**

参数说明

**input_excel**： video_report.csv文件

**output_path**：为输出文件路径 生成video_pull_report.csv文件

会自动跳过已经获取的任务, 删除根目录cache_data/VideoPullGenerator的文件可以重新获取

根据需要配置线程

**num_threads**: 线程数,默认2

In [25]:
task_video_csv = f'{output_data_dir}/video_report.csv'
output_path = output_data_dir
command = f"python -m zhipuai_platform_video.video_pull --task_video_csv {task_video_csv} --output_path {output_path}"
!{command}

VideoPullGenerator:VideoPullGenerator-4709c5b91cf16cc27cb145fd46cefa03 {'video_task_id': '1000088875824486025586417'}
2024-07-29 15:08:04,422 - httpx - INFO - HTTP Request: GET https://open.bigmodel.cn/api/paas/v4/async-result/1000088875824486025586417 "HTTP/1.1 200 OK"
VideoPullGenerator:VideoPullGenerator-80e181867c6209c17c22a712a0ea85c5 {'video_task_id': '1000088875819916179994141'}
2024-07-29 15:08:04,582 - httpx - INFO - HTTP Request: GET https://open.bigmodel.cn/api/paas/v4/async-result/1000088875819916179994141 "HTTP/1.1 200 OK"


# 6.查看视频效果

通过video_pull_report_csv中的链接可以预览视频，这在您本地运行之后可以正常查看。

In [26]:
import pandas as pd
from IPython.display import display, HTML

video_pull_report_csv = f'{output_data_dir}/video_pull_report.csv'

# Load the data

video_pull_report = pd.read_csv(video_pull_report_csv)
 
# Extract video URLs from the 'result' column
video_pull_report['video_url'] = video_pull_report['result'].apply(lambda x: eval(x)[0]['url'])

# Display videos with their IDs
for index, row in video_pull_report.iterrows():
    video_id = row['video_task_id']
    video_url = row['video_url']
    display(HTML(f'<h3>Video ID: {video_id}</h3><video width="400" controls><source src="{video_url}" type="video/mp4">Your browser does not support the video tag.</video>'))